# Projeto Final - Codenation Data Science - JOINVILLE

# SQUAD 1 - Objetivo

O objetivo deste produto é fornecer um serviço automatizado que recomenda leads para um usuário dado sua atual lista de clientes (Portfólio).

# Contextualização
Algumas empresas gostariam de saber quem são as demais empresas em um determinado mercado (população) que tem maior probabilidade se tornarem seus próximos clientes. Ou seja, a solução deve encontrar no mercado quem são os leads mais aderentes dado as características dos clientes presentes no portfólio do usuário.

Além disso, a solução deve ser agnóstica ao usuário. Qualquer usuário com uma lista de clientes que queira explorar esse mercado pode extrair valor do serviço.

Para o desafio, deverão ser consideradas as seguintes bases:

Mercado: Base com informações sobre as empresas do Mercado a ser considerado. Portfolio 1: Ids dos clientes da empresa 1 Portfolio 2: Ids dos clientes da empresa 2 Portfolio 3: Ids dos clientes da empresa 3

Obs: todas as empresas(ids) dos portfolios estão contidos no Mercado(base de população).

As bases de portfólio poderão ser utilizadas para testar a aderência da solução. Além disso, se a equipe desejar, poderá simular portfólios por meio de amostragens no mercado.

Premissas:

1. Não serão considerados os dados de "coligados"
2. Não serão considerado o relacionamento entre filial/matriz, cada cliente será avaliado individualmente

In [1]:
#Importando as bibliotecas
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns

In [86]:
from IPython.core.pylabtools import figsize

figsize(12, 8)

sns.set()

## Importando os Arquivos

In [90]:
#Importando o Dataframe de Mercado e eliminando os dados 
mercado_orig = pd.read_csv('estaticos_market.csv')
mercado = mercado_orig.copy()

# Não serão considerados os dados do grupo
mercado = mercado.drop(['Unnamed: 0'] , axis='columns')
mercado = mercado.drop(['de_faixa_faturamento_estimado_grupo'] , axis='columns')
mercado = mercado.drop(['vl_faturamento_estimado_aux'] , axis='columns')
mercado = mercado.drop(['vl_faturamento_estimado_grupo_aux'] , axis='columns')

# Não serão considerados os dados relacionados as contratações dos funcionarios no periodo - Apenas a quantidade atual de funcionários da empresa
mercado = mercado.drop(['meses_ultima_contratacaco'] , axis='columns')
mercado = mercado.drop(['qt_admitidos_12meses'] , axis='columns')
mercado = mercado.drop(['qt_desligados_12meses'] , axis='columns')
mercado = mercado.drop(['qt_desligados'] , axis='columns')
mercado = mercado.drop(['qt_admitidos'] , axis='columns')
mercado = mercado.drop(['qt_funcionarios_12meses'] , axis='columns')
mercado = mercado.drop(['qt_funcionarios_24meses'] , axis='columns')
mercado = mercado.drop(['tx_crescimento_12meses'] , axis='columns')
mercado = mercado.drop(['tx_crescimento_24meses'] , axis='columns')
mercado = mercado.drop(['tx_rotatividade'] , axis='columns')
mercado = mercado.drop(['min_meses_servicos'] , axis='columns')
mercado = mercado.drop(['min_meses_servicos_all'] , axis='columns')
mercado = mercado.drop(['max_meses_servicos_all'] , axis='columns')
mercado = mercado.drop(['grau_instrucao_macro_desconhecido'] , axis='columns')
mercado = mercado.drop(['total'] , axis='columns')

# Não serão considerados os valores relacionados as coligadas
lista_colig = [x for x in mercado.columns if 'colig' in x ]
mercado = mercado.drop(lista_colig , axis='columns')

# Não serão considerados os dados relacionados aos sócios - Apenas a quantidade atual de sócios
lista_socio = [x for x in mercado.columns if 'socio' in x ]
lista_socio.remove('qt_socios')
mercado = mercado.drop(lista_socio , axis='columns')

# Não serão considerados os dados de Ramo, Segmento e Divisão - Consideraremos o dado de setor para agrupamento dos clientes
mercado = mercado.drop(['de_ramo'] , axis='columns')
mercado = mercado.drop(['nm_segmento'] , axis='columns')
mercado = mercado.drop(['nm_divisao'] , axis='columns')

In [91]:
#Importando o Dataframe de Portfolio1
portfolio1 = pd.read_csv('estaticos_portfolio1.csv')
portfolio1 = portfolio1.drop(['Unnamed: 0'] , axis='columns')

In [92]:
#estaticos_portfolio1 o Dataframe de Portfolio2
portfolio2 = pd.read_csv('estaticos_portfolio2.csv')
portfolio2 = portfolio2.drop(['Unnamed: 0'] , axis='columns')
portfolio2 = pd.merge(portfolio2, mercado, on='id', how='inner')

In [93]:
#Importando o Dataframe de Portfolio3
portfolio3 = pd.read_csv('estaticos_portfolio3.csv')
portfolio3 = portfolio3.drop(['Unnamed: 0'] , axis='columns')
portfolio3 = pd.merge(portfolio3, mercado, on='id', how='inner')

## Iniciando as Análises dos Arquivos

Nesse momento avaliaremos os dados do dataframe

In [94]:
# 5 Primeiras Linhas
mercado.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,media_meses_servicos_all,media_meses_servicos,max_meses_servicos,qt_funcionarios,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,...,1.0,NaN,1.0,25.0,NaN,13.309195,43.738462,93.266667,26.0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVIÇO,1.463014,1 a 5,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVIÇO,7.093151,5 a 10,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVIÇO,6.512329,5 a 10,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,SERVIÇO,3.200000,1 a 5,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [95]:
#Verificando Quantidade de Colunas e Observações
mercado.shape

(462298, 70)

In [56]:
#Verificando estatisticas
mercado.describe()

,idade_empresa_anos,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
count,462298.000000,280.000000,176.000000,30684.000000,30684.000000,6590.000000,460371.000000,460371.000000,1760.000000,336.000000,...,76261.000000,76261.000000,76261.000000,103574.000000,103574.000000,103574.000000,73888.000000,74014.000000,103574.000000,462298.000000
mean,9.948677,32.014286,3.818182,2.244329,1.177813,5.769044,3.591801,48.699177,134.597159,15.934524,...,58.060498,96.661983,36.258851,12.324570,12.178529,14.343329,3.097607,-5.834288,9.510699,28.737044
std,9.615664,81.280168,6.797555,9.572536,6.589059,25.450950,72.600352,1206.696804,683.774506,29.708663,...,142.951278,279.541243,123.411370,222.456579,222.584458,239.885359,163.581549,443.825819,27.918737,468.626094
min,0.016438,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,...,1.933333,1.933333,1.933333,0.000000,0.000000,0.000000,-100.000000,-100.000000,0.000000,0.000000
25%,2.756164,15.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,15.000000,1.000000,...,25.203704,33.333333,6.966667,0.000000,0.000000,0.000000,0.000000,-44.444444,0.000000,0.000000
50%,6.704110,15.000000,2.000000,1.000000,0.000000,2.000000,0.000000,0.000000,15.000000,3.000000,...,43.533333,61.766667,23.200000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000
75%,14.465753,30.000000,4.000000,2.000000,1.000000,4.000000,0.000000,0.000000,66.250000,8.000000,...,68.883333,96.266667,46.500000,5.000000,5.000000,6.000000,0.000000,0.000000,0.000000,0.000000
max,106.432877,1215.000000,50.000000,489.000000,363.000000,1017.000000,9782.000000,122090.000000,11922.000000,108.000000,...,5099.066667,5099.066667,5099.066667,51547.000000,51547.000000,54205.000000,27800.000000,87300.000000,1400.000000,9647.000000


In [57]:
#Verificando a quantidade valores nulos por coluna
mercado.isnull().sum()

id                                                    0
fl_matriz                                             0
de_natureza_juridica                                  0
sg_uf                                                 0
natureza_juridica_macro                               0
de_ramo                                               0
setor                                              1927
idade_empresa_anos                                    0
idade_emp_cat                                         0
fl_me                                                 0
fl_sa                                                 0
fl_epp                                                0
fl_mei                                                0
fl_ltda                                               0
dt_situacao                                           6
fl_st_especial                                        0
fl_email                                              0
fl_telefone                                     

In [79]:
#Avaliação do mercado que estão sem setor
mercado.groupby('setor').id.count()

setor
AGROPECUARIA          3247
COMERCIO            211224
CONSTRUÇÃO CIVIL     25363
INDUSTRIA            30893
SERVIÇO             189644
Name: id, dtype: int64

In [69]:
#Avaliação do Ramo que estão sem setor
mercado.groupby('de_ramo').id.count()

de_ramo
ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL      2754
ATIVIDADES IMOBILIARIAS                                2670
BENS DE CONSUMO                                       21227
COMERCIO E REPARACAO DE VEICULOS                      22356
COMERCIO POR ATACADO                                  16464
COMERCIO VAREJISTA                                   172404
CULTURA, ESPORTE E RECREACAO                           5503
ELETRICIDADE E GAS                                      607
ELETROELETRONICOS                                       251
FARMACEUTICA                                             22
INDUSTRIA AUTOMOTIVA                                    410
INDUSTRIA DA CONSTRUCAO                               25363
INDUSTRIA DIGITAL                                      1849
INDUSTRIAS EXTRATIVAS                                   896
ORGANISMOS INTERNACIONAIS                                 9
OUTROS                                                 1927
PAPEL E CELULOSE                

In [78]:
mercado.groupby('nm_segmento').id.count()

nm_segmento
ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL                       2754
AGRICULTURA PECUARIA PRODUCAO FLORESTAL PESCA E AQUICULTURA            3247
AGUA ESGOTO ATIVIDADES DE GESTAO DE RESIDUOS E DESCONTAMINACAO         1238
ALOJAMENTO E ALIMENTACAO                                              30180
ARTES CULTURA ESPORTE E RECREACAO                                      5503
ATIVIDADES ADMINISTRATIVAS E SERVICOS COMPLEMENTARES                  21326
ATIVIDADES FINANCEIRAS DE SEGUROS E SERVICOS RELACIONADOS              3287
ATIVIDADES IMOBILIARIAS                                                2670
ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS                       17371
COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E MOTOCICLETAS           211224
CONSTRUCAO                                                            25363
EDUCACAO                                                              11311
ELETRICIDADE E GAS                                                      607


In [80]:
mercado.groupby('nm_divisao').id.count()

nm_divisao
ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL                                                                             2754
AGENCIAS DE VIAGENS OPERADORES TURISTICOS E SERVICOS DE RESERVAS                                                             2308
AGRICULTURA PECUARIA E SERVICOS RELACIONADOS                                                                                 2099
ALIMENTACAO                                                                                                                 27540
ALOJAMENTO                                                                                                                   2640
ALUGUEIS NAO IMOBILIARIOS E GESTAO DE ATIVOS INTANGIVEIS NAO FINANCEIROS                                                     3367
ARMAZENAMENTO E ATIVIDADES AUXILIARES DOS TRANSPORTES                                                                        1615
ATIVIDADES ARTISTICAS CRIATIVAS E DE ESPETACULOS                               

In [31]:
#Distribuição de Setor do portfolio1
portfolio1.setor.unique()

array(['COMERCIO', 'INDUSTRIA', 'SERVIÇO'], dtype=object)

In [75]:
#Avaliação de Distribuição de setor do portfolio1
portfolio1.groupby('setor').id.count()

setor
COMERCIO     496
INDUSTRIA     52
SERVIÇO        7
Name: id, dtype: int64

In [76]:
#Avaliação de Distribuição de setor do portfolio1
portfolio1.groupby('de_ramo').id.count()

de_ramo
BENS DE CONSUMO        59
COMERCIO VAREJISTA    496
Name: id, dtype: int64

In [33]:
#Avaliando industria
portfolio1[portfolio1.setor == 'INDUSTRIA']

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
10,d37003c6e5d299c3fcd45f36ec6c26d39770f1d52edc41...,True,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,3.756164,1 a 5,False,...,23.200000,23.200000,23.200000,1.0,1.0,0.0,0.000000,NaN,0.000000,0
13,fa2a153f8449584ae3f0e8252a56fa4309509506c921c3...,True,EMPRESARIO INDIVIDUAL,MA,OUTROS,BENS DE CONSUMO,INDUSTRIA,20.572603,> 20,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
32,26bdeb6de15f66ce158b7ddd473860352b48e7693be2ed...,False,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,43.186301,> 20,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
36,e5aeb8197c61261b80d66f1da4e3760e240a1f5557208a...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,BENS DE CONSUMO,INDUSTRIA,23.731507,> 20,False,...,50.895238,234.266667,8.033333,7.0,7.0,10.0,0.000000,-30.000000,28.571429,0
37,d4cb950ffc7f6de0a38403a590bff553a03185b1ec9dac...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,16.350685,15 a 20,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
40,3149cee2c71cee081f981fdf2c380c25196d4573fda435...,True,EMPRESARIO INDIVIDUAL,RO,OUTROS,BENS DE CONSUMO,INDUSTRIA,5.405479,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
55,5e0dc52277e30c08cf1f616ee753fabc986b8c29710250...,True,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,10.430137,10 a 15,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
68,b5065c017558e45d9fe978a8392171eba7bb3d42f4531c...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,BENS DE CONSUMO,INDUSTRIA,0.367123,<= 1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
74,187f3b0cd730f78d0535b294b5043a2110fe50bd51b1f5...,True,EMPRESARIO INDIVIDUAL,RN,OUTROS,BENS DE CONSUMO,INDUSTRIA,1.336986,1 a 5,False,...,14.452564,16.100000,4.800000,52.0,55.0,6.0,-5.454545,766.666667,16.981132,0
78,911cdae516f43beed1e7e6e8b7781adeb306f03e1246c8...,True,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,12.876712,10 a 15,False,...,61.766667,61.766667,61.766667,1.0,1.0,4.0,0.000000,-75.000000,0.000000,0


In [34]:
#Avaliando industria
portfolio1[portfolio1.setor == 'SERVIÇO']

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
138,0c7fc91f2a93428c54775ca70662b2727d1af510e4e43b...,True,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,AM,OUTROS,BENS DE CONSUMO,SERVIÇO,18.882192,15 a 20,False,...,50.873333,95.233333,13.766667,5.0,6.0,10.0,-16.666667,-50.0,18.181818,0
140,bee92f7d793051acada2100e920a7b77b97acb38340de4...,True,EMPRESARIO INDIVIDUAL,AC,OUTROS,BENS DE CONSUMO,SERVIÇO,7.638356,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
171,950867d5aa9a5a82a5eaec4e939c84543b8ea3d3b41fc6...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,BENS DE CONSUMO,SERVIÇO,4.408219,1 a 5,False,...,49.600000,49.600000,49.600000,1.0,1.0,1.0,0.000000,0.0,0.000000,0
173,f6464ca3623913eaee1186acc183e965fa0553bf636300...,True,SOCIEDADE EMPRESARIA LIMITADA,PI,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,SERVIÇO,5.079452,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
234,724af49650e12e7c0f3838672dcdec4558d51984a38c02...,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,BENS DE CONSUMO,SERVIÇO,1.991781,1 a 5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
426,6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c78...,True,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,SERVIÇO,6.361644,5 a 10,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
432,6c1ff3d6e64d01dcd1f8d435167f8775ab8867978b1b20...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,BENS DE CONSUMO,SERVIÇO,2.695890,1 a 5,False,...,27.266667,27.266667,27.266667,1.0,1.0,1.0,0.000000,0.0,0.000000,0


In [35]:
#Distribuição de Setor do portfolio2
portfolio2.setor.unique()

array(['SERVIÇO', 'AGROPECUARIA', 'COMERCIO', 'CONSTRUÇÃO CIVIL',
       'INDUSTRIA'], dtype=object)

In [36]:
#Distribuição de Setor do portfolio3
portfolio3.setor.unique()

array(['SERVIÇO'], dtype=object)